In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import datetime as dt
import alpaca_trade_api as tradeapi
import json
from alpaca_trade_api.rest import REST, TimeFrame
load_dotenv()

True

In [2]:
alpaca_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

tradeapi = REST(alpaca_key, alpaca_secret_key)

In [3]:
def get_daily_trade_data(ticker, start, end, tradeapi): 
    ticker_df = tradeapi.get_bars(
        ticker,
        TimeFrame.Day,
        start,
        end
    ).df
    ticker_df['ticker'] = ticker

    return ticker_df

In [4]:
get_daily_trade_data('HD', '2022-01-03', '2022-01-07', tradeapi)

,open,high,low,close,volume,trade_count,vwap,ticker
timestamp,,,,,,,,
2022-01-03 05:00:00+00:00,416.57,417.8446,403.26,408.64,3720356,85021,408.261522,HD
2022-01-04 05:00:00+00:00,408.64,413.6100,408.19,412.84,3264609,67397,411.849521,HD
2022-01-05 05:00:00+00:00,414.00,414.2200,406.81,407.24,4419021,84372,409.332968,HD
2022-01-06 05:00:00+00:00,408.56,409.0700,403.02,405.76,3307417,68950,405.615925,HD
2022-01-07 05:00:00+00:00,403.36,404.0900,392.88,393.61,4514133,89211,395.971181,HD


In [5]:
ticker_list = pd.read_csv('./Data/Cleaned_Data/Ticker_library.csv')['Ticker'].to_list()

def make_tickers_df(ticker_list, start_date_str, end_date_str, tradeapi):
    ticker_dfs_list = [get_daily_trade_data(ticker, start_date_str, end_date_str, tradeapi) for ticker in ticker_list]
    tickers_df = pd.concat(ticker_dfs_list, axis=0, join='outer')
    tickers_df.index = tickers_df.index.date
    tickers_df = tickers_df[['ticker','close','volume']]
    return tickers_df
    
    #['symbol','close'].reset_index().rename(columns={'timestamp':'date'}).set_index('date').dt.date
    #return tickers_df.index.dt.date.reset_index().rename(columns={"timestamp":"date"})

In [6]:
stocks_df = make_tickers_df(ticker_list, '2012-06-01', '2022-06-01', tradeapi)
stocks_df.head()

,ticker,close,volume
2015-12-01,NFLX,125.37,12550755
2015-12-02,NFLX,128.93,24105712
2015-12-03,NFLX,126.81,26644540
2015-12-04,NFLX,130.93,20250411
2015-12-07,NFLX,125.36,23351531


In [7]:
def make_tickers_dict(ticker_list, start_date_str, end_date_str, tradeapi):
    return {ticker: get_daily_trade_data(ticker, start_date_str, end_date_str, tradeapi) for ticker in ticker_list}

In [8]:
ticker_list = ['AAPL', 'XOM']
my_tick_dict = make_tickers_dict(ticker_list, '2022-01-03', '2022-01-07', tradeapi)
my_tick_dict

{'AAPL':                              open    high     low    close     volume  \
 timestamp                                                               
 2022-01-03 05:00:00+00:00  177.81  182.88  177.71  182.010  104690466   
 2022-01-04 05:00:00+00:00  182.61  182.94  179.12  179.700   99284337   
 2022-01-05 05:00:00+00:00  179.61  180.17  174.65  174.835   86753815   
 2022-01-06 05:00:00+00:00  172.40  175.30  171.64  172.000   96792639   
 2022-01-07 05:00:00+00:00  172.73  174.14  171.03  172.170   87682425   
 
                            trade_count        vwap ticker  
 timestamp                                                  
 2022-01-03 05:00:00+00:00       772367  181.415512   AAPL  
 2022-01-04 05:00:00+00:00       831619  180.555450   AAPL  
 2022-01-05 05:00:00+00:00       844519  177.497784   AAPL  
 2022-01-06 05:00:00+00:00       958689  173.079885   AAPL  
 2022-01-07 05:00:00+00:00       716119  172.439116   AAPL  ,
 'XOM':                             open    

## News Article Vader Analyzer

In [9]:
from libs.krista_lib import form_df

from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\altma\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [12]:
api_key = os.getenv("NEWSAPI_KEY")
newsapi = NewsApiClient(api_key=api_key)
print(type(api_key))


<class 'str'>


In [13]:
newsapi.get_top_headlines(q='Diamondback Energy Inc.', language='en')

{'status': 'ok', 'totalResults': 0, 'articles': []}

In [14]:
from libs.krista_lib import get_news
get_news('Diamondback Energy Inc.')

{'status': 'ok',
 'totalResults': 33,
 'articles': [{'source': {'id': None, 'name': 'Yahoo Entertainment'},
   'author': 'Diamondback Energy, Inc.',
   'title': 'Diamondback Energy, Inc. Announces Agreement to Acquire Publicly Held Units of Rattler Midstream LP',
   'description': 'MIDLAND, Texas, May 16, 2022 (GLOBE NEWSWIRE) -- Diamondback Energy, Inc. (NASDAQ: FANG) (“Diamondback”) and Rattler Midstream LP (NASDAQ: RTLR) (“Rattler”) ...',
   'url': 'https://finance.yahoo.com/news/diamondback-energy-inc-announces-agreement-100000774.html',
   'urlToImage': 'https://s.yimg.com/ny/api/res/1.2/QCPdr31b3fVX9ECEBihSHw--/YXBwaWQ9aGlnaGxhbmRlcjt3PTEyMDA7aD00Mzg-/https://s.yimg.com/uu/api/res/1.2/elJVLYi5uFmdVMCgitE2Ug--~B/aD04NzY7dz0yNDAwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/globenewswire.com/65079ff8af771091c3a6d0b503fecdcc',
   'publishedAt': '2022-05-16T10:00:00Z',
   'content': 'MIDLAND, Texas, May 16, 2022 (GLOBE NEWSWIRE) -- Diamondback Energy, Inc. (NASDAQ: FANG) (Diamondba

In [19]:
company_df = pd.read_csv('./Data/Cleaned_Data/Ticker_library.csv')
company_df = company_df.rename(columns={'Company Name': 'Company_Name'})
company_dict = dict(zip(company_df.Company_Name, company_df.Ticker))
company_list = list(company_dict.keys())
tickers_list = company_dict.values()
ls = [type(item) for item in company_list]
print(ls)
tickers_list


[<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>]


dict_values(['NFLX', 'FB', 'UBER', 'MCHP', 'ABNB', 'FANG', 'MRO', 'DVN', 'SPWR', 'REGI', 'MTRX', 'BLK', 'PYPL', 'MELI', 'SOFI'])

In [ ]:
df = make_tickers_df(tickers_list, '2012-06-01', '2022-06-01', tradeapi)
df

In [23]:
df.to_csv('../Notebooks/Data/Cleaned_Data/stock_data.csv')

In [ ]:
api_key = os.getenv("NEWSAPI_KEY")

newsapi = NewsApiClient(api_key=api_key)

#Get news articles on certain topic based on keywords
def get_news(keywords):  
    news_article = newsapi.get_everything(
            q = keywords, language='en', sort_by= 'relevancy'
    )
    return news_article

#Creates dataframe of the articles chosen 
def form_df(keywords):
    news = get_news(keywords)['articles']

    articles = []
    for article in news:
        try:
            title = article['title']
            description = article['description']
            text = article['content']
            date = article['publishedAt'][:10]

            articles.append({
                'date' : date,
                'ticker': keyword.apply(lambda x: company_dict[x]),
                'text' : text,
                'title' : title,
                'description' : description,
                'language' : 'en'
            })
        except AttributeError:
            pass
    
    return pd.DataFrame(articles)

In [16]:
def news_vader(company_dict):
    company_list = company_dict.keys()

    df_list = []
    for company in company_list:
        try:
            df_list.append(form_df(company))
        except:
            continue
    news_df = pd.concat(df_list, axis=0, join='outer')
    
    return news_df



In [ ]:
import libs.krista_lib as news
# News Handling
News_allTickers = [news.form_df(ticker) for ticker in tickers]

In [17]:
news_vader(company_dict)

,title,description,text,date,language
0,Netflix rolls out shuffle button for kids,Picking out something new to watch from Netfli...,Picking out something new to watch from Netfli...,2022-05-19,en
1,Netflix is reportedly looking into livestreaming,Netflix is reportedly looking into airing live...,The platform could add the capability for come...,2022-05-14,en
2,Will Netflix Survive the Streaming Wars?,The streaming wars are no longer about which a...,The streaming wars are no longer about which a...,2022-06-02,en
3,How to see everything you've watched on Netfli...,Streaming is a curious beast. One minute you'l...,Streaming is a curious beast. One minute you'l...,2022-05-10,en
4,Netflix’s ad-supported tier and password shari...,Netflix is speeding up its plan to launch ad-s...,The timetables are accelerating\r\nIllustratio...,2022-05-10,en
...,...,...,...,...,...
15,: You Bought. They Sold. Meet some of the insi...,"The stocks of FIGS, Coinbase, Coupang and Robi...","A year ago, Heather Hasson and Catherine Trina...",2022-05-18,en
16,SoFi Is Down 66% This Year. Is It Time to Buy?,The digital banking company has gotten crushed...,Like many other tech and fintech stocks this y...,2022-05-14,en
17,"Why Shares of Affirm, SoFi, and LendingClub Ar...",Fintech stocks are taking a hit after Upstart'...,What happened\r\nShares of several consumer fi...,2022-05-10,en
18,"Down Almost 70%, SoFi Technologies' CEO Keeps ...",Is SoFi's CEO sending a signal to investors th...,Investor sentiment has cratered in recent time...,2022-05-19,en


In [ ]:
form_df(company_list)

TypeError: keyword/phrase q param should be of type str